<a href="https://colab.research.google.com/github/migub/recommender-systems/blob/main/Notebooks/04_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Logistic Regression


In [ ]:
## 1. Setup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import joblib
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## 2. Data Loading
df = pd.read_csv("/content/drive/MyDrive/Recommender_Systems/train_preprocessed.csv")

In [ ]:
## 3. Prepare Training Data
# Treat as category types
df['platform_name'] = df['platform_name'].astype('category')
df['platform_family'] = df['platform_family'].astype('category')
df['listen_type'] = df['listen_type'].astype('category')
df['user_gender'] = df['user_gender'].astype('category')
df['genre_id'] = df['genre_id'].astype('category')
df['media_id'] = df['media_id'].astype('category')
df['album_id'] = df['album_id'].astype('category')
df['user_id'] = df['user_id'].astype('category')
df['artist_id'] = df['artist_id'].astype('category')

# Drop date columns
df = df.drop(columns=['release_date'], errors='ignore')
df = df.drop(columns=['ts_listen'], errors='ignore')

# Display basic info
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152705 entries, 0 to 7152704
Data columns (total 15 columns):
 #   Column            Dtype   
---  ------            -----   
 0   genre_id          category
 1   media_id          category
 2   album_id          category
 3   context_type      int64   
 4   platform_name     category
 5   platform_family   category
 6   media_duration    float64 
 7   listen_type       category
 8   user_gender       category
 9   user_id           category
 10  artist_id         category
 11  user_age          float64 
 12  is_listened       bool    
 13  listen_hour       int64   
 14  listen_dayofweek  int64   
dtypes: bool(1), category(9), float64(2), int64(3)
memory usage: 435.4 MB


,genre_id,media_id,album_id,context_type,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened,listen_hour,listen_dayofweek
0,25471,222606,41774,12,1,0,0.491379,0,0,9241,55164,0.916667,False,13,3
1,25571,250467,43941,0,2,1,0.267241,0,0,16547,55830,1.000000,True,22,2
2,16,305197,48078,1,2,1,0.172414,1,1,7665,2704,0.916667,True,13,5
3,7,900502,71521,0,0,0,0.564655,0,1,1580,938,1.000000,False,9,5
4,7,542335,71718,0,0,0,0.176724,0,1,1812,2939,0.500000,True,18,5


In [ ]:
## 4. Train-test split
X = df.drop(columns=['is_listened'])
y = df['is_listened']

# Split `train.csv` into train (80%) and validation (20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
## 5. Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

Model saved to: /content/drive/My Drive/Recommender_Systems/deezer_logistic_model.pkl


In [ ]:
## 6. Evaluate model on validation set
val_predictions = model.predict(X_val)
val_accuracy = (val_predictions == y_val).mean()
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Accuracy: 0.6830


In [ ]:
## 7. Train final model on full `train.csv`
model.fit(X, y)

# Define path to save the model in Google Drive
model_path = "/content/drive/My Drive/Recommender_Systems/deezer_logistic_model.pkl"

# Save the trained model
joblib.dump(model, model_path)

print(f"Model saved to: {model_path}")

Model saved to: /content/drive/My Drive/Recommender_Systems/deezer_logistic_model.pkl


In [ ]:
## 8. (Optional) Predict on Kaggle's `test.csv`
# Ensure `test.csv` has the same features as `train.csv`
X_test = test.drop(columns=['sample_id'], errors='ignore')  # Remove sample_id

# Predict on Kaggle's `test.csv`
predictions = model.predict_proba(X_test)[:, 1]  # Get probability of `is_listened`

# Convert to binary predictions (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Save predictions for Kaggle submission
submission = pd.DataFrame({'id': test['sample_id'], 'is_listened': binary_predictions})  # Use sample_id for Kaggle submission
submission.to_csv("submission.csv", index=False)

print("Submission file saved as submission.csv")

Submission file saved as submission.csv
